# **Airport Optimization Problem - QUBO**
# _____________________________________________


## Problem Formulation
We will optimize flight assignments to gates at an airport using PyQubo and Dynex. Our goal is to minimize conflicts and efficiently manage gate assignments considering various constraints.

#### **Constraints**
1. **Gate Availability**: Each flight must be assigned to an available gate, with no overlaps in time.
2. **Flight Timing**: Flights must be scheduled without conflicts, with at least a one-hour gap between consecutive flights at the same gate if possible.
3. **Parking Position Constraints**: Each flight should be assigned to a parking position based on the apron type and availability.
4. **Flight Conflicts**: Avoid conflicts where multiple flights are assigned to the same parking position at the same time.


# _____________________________________________

###  **1: Import Libraries**

In [1]:
# Import necessary libraries for quantum optimization and data handling
import dynex
from pyqubo import Binary  
import pandas as pd  
from datetime import datetime  

# Import libraries for building and running Dash applications
import dash  
from dash import dcc, html  
from dash.dependencies import Input, Output  

# Import Plotly and Matplotlib for data visualization
import plotly.graph_objects as go  
import matplotlib.pyplot as plt  
import matplotlib.patches as patches  

# Import additional libraries for handling geographic data and images
import numpy as np  
import io  
import base64  
from shapely.geometry import Point, LineString, box  
import osmnx as ox  
import matplotlib.image as mpimg  
from matplotlib.offsetbox import OffsetImage, AnnotationBbox 
from matplotlib.lines import Line2D  
from datetime import datetime  

### **2. Airport Data Processing and Gate Classification**
#### _____________________________________________
# JNB_DATA (Johannesburg Airport Data)


##### Define international Aiports locations data

In [2]:
locations = {
    'Johannesburg (JNB)': (-26.1337, 28.2421),
    'Ellisras (ELL)': (-23.6667, 27.7333),
    'Margate (MGH)': (-30.857, 30.343),
    'Plettenberg Bay (PBZ)': (-34.0889, 23.325),
    'Mumbai (BOM)': (19.0896, 72.8656),
    'London (LHR)': (51.4775, -0.4614),
    'Paris (CDG)': (49.0097, 2.5479),
    'Abu Dhabi (AUH)': (24.4539, 54.3773),
    'New York (JFK)': (40.6413, -73.7781),
    'Cape Town (CPT)': (-33.9698, 18.5976),
    'Durban (DUR)': (-29.8587, 31.0218),
    'Harare (HRE)': (-17.9318, 31.0928),
    'Livingstone (LVI)': (-17.8216, 25.8224),
    'Mauritius (MRU)': (-20.3484, 57.5522),
    'Port Elizabeth (PLZ)': (-33.965, 25.6001),
    'Victoria Falls (VFA)': (-18.0959, 25.839),
    'Windhoek (WDH)': (-22.4799, 17.4709),
    'Kasane (BBK)': (-17.8328, 25.1623),
    'Francistown (FRW)': (-21.1596, 27.5079),
    'Gaborone (GBE)': (-24.6282, 25.9231),
    'Mubai (MUB)': (-19.9726, 23.4311),
    'Lubumbashi (FBM)': (-11.6696, 27.4585),
    'Beijing (PEK)': (40.0801, 116.5846),
    'Hong Kong (HKG)': (22.3193, 114.1694),
    'Ndola (NLA)': (-12.9984, 28.6594),
    'Atlanta (ATL)': (33.749, -84.388),
    'Luanda (LAD)': (-8.839, 13.2894),
    'Dubai (DXB)': (25.2532, 55.3657),
    'Addis Ababa (ADD)': (9.02, 38.7469),
    'Blantyre (BLZ)': (-15.7861, 35.0058),
    'Victoria (SEZ)': (-4.6796, 55.4917),
    'George (GRJ)': (-34.0056, 22.378),
    'Zanzibar (ZNZ)': (-6.1659, 39.2026),
    'São Paulo (GRU)': (-23.5505, -46.6333),
    'Amsterdam (AMS)': (52.3676, 4.9041),
    'Nairobi (NBO)': (-1.286389, 36.817223),
    'Frankfurt (FRA)': (50.0379, 8.5622),
    'Munich (MUC)': (48.3538, 11.7861),
    'Zurich (ZRH)': (47.4647, 8.5492),
    'Palastine (PLA)': (32.0853, 34.7818),
    'Antananarivo (TNR)': (-18.8792, 47.5079),
    'East London (ELS)': (-33.0292, 27.8546),
    'Cairo (CAI)': (30.0444, 31.2357),
    'Singapore (SIN)': (1.3644, 103.9915),
    'Perth (PER)': (-31.9505, 115.8605),
    'Sydney (SYD)': (-33.8688, 151.2093),
    'Doha (DOH)': (25.276987, 51.520008),
    'Maputo (MPM)': (-25.9208, 32.5732),
    'Accra (ACC)': (5.6037, -0.187),
    'Apala (APL)': (-15.1056, 39.2818),
    'Beira (BEW)': (-19.7964, 34.9076),
    'Bloemfontein (BFN)': (-29.0852, 26.1596),
    'Bangkok (BKK)': (13.689999, 100.750112),
    'Bulawayo (BUQ)': (-20.0174, 28.6179),
    'Brazzaville (BZV)': (-4.2634, 15.2429),
    'Dar es Salaam (DAR)': (-6.7924, 39.2083),
    'Dakar (DKR)': (14.6711, -17.0661),
    'Entebbe (EBB)': (0.0423, 32.4431),
    'Kinshasa (FIH)': (-4.4419, 15.2663),
    'Hoedspruit (HDS)': (-24.3686, 31.0488),
    'Kigali (KGL)': (-1.9686, 30.1395),
    'Kimberley (KIM)': (-28.7384, 24.7639),
    'Libreville (LBV)': (0.4162, 9.4673),
    'Lilongwe (LLW)': (-13.963, 33.7741),
    'Lagos (LOS)': (6.5779, 3.3212),
    'Lusaka (LUN)': (-15.3875, 28.3228),
    'Mpumalanga (MQP)': (-25.3844, 31.1056),
    'Masvingo (MSU)': (-29.3158, 27.4869),
    'Mtata (MTS)': (-26.5296, 31.3802),
    'Phalaborwa (PHW)': (-23.9377, 31.1411),
    'Pointe-Noire (PNR)': (-4.7809, 11.8635),
    'Polo (POL)': (-12.973, 40.5178),
    'Pietersburg (PTG)': (-23.8962, 29.4486),
    'Pietermaritzburg (PZB)': (-29.6006, 30.3794),
    'Richards Bay (RCB)': (-28.774, 32.0604),
    'Tete (TET)': (-16.1048, 33.6402),
    'Upington (UTN)': (-28.456, 21.2418),
    'Umtata (UTT)': (-31.6018, 28.7746),
    'Vanga (VNX)': (-22.0184, 35.3133),
    'Walvis Bay (WVB)': (-22.9799, 14.6453),
    'Jeddah (JED)': (21.6796, 39.1565),
    'Istanbul (IST)': (41.275278, 28.751944),
    'Inhambane (INH)': (-23.8764, 35.4085),
    'Saint-Denis (RUN)': (-20.8871, 55.5103)
}

##### **Location Data:** Contains airport coordinates, converted to a DataFrame.

In [3]:
# Convert locations to DataFrame
airportsData = pd.DataFrame({
    'airportCode': [key.split(' ')[-1].strip('()') for key in locations.keys()],
    'latitude': [value[0] for value in locations.values()],
    'longitude': [value[1] for value in locations.values()]
})

# Extract Source and Destination from Flight ID
def ExtractSourceDestination(flightId):
    parts = flightId.split('_')
    if len(parts) == 4:
        source = parts[2]
        destination = parts[3]
        return source, destination
    return None, None


##### **Gate and Parking Position Classification:** Functions are defined to classify and process gate and parking position data.

In [4]:
# Function to classify parking positions based on 'ref' as defined at OpenStreetMap
def ClassifyParkingPosition(ref):
    if pd.notna(ref):
        if 'A1R' in ref:
            return 'Alpha Apron'
        elif ref.startswith('A') and ref[1:].isdigit():
            num = int(ref[1:])
            if 4 <= num <= 13:
                return 'Airport Apron'
            else:
                return 'Alpha Apron'
        elif ref.startswith('C'):
            return 'Charlie Apron'
        elif ref.startswith('E'):
            return 'Echo Apron'
    return 'Unknown'

# Function to plot smaller triangles for parking positions
def PlotTriangle(ax, x, y, color='blue'):
    triangleSize = 0.00005 
    triangle = patches.Polygon(
        [[x, y], [x - triangleSize, y - triangleSize], [x + triangleSize, y - triangleSize]],
        closed=True, color=color
    )
    ax.add_patch(triangle)
  
def SplitGates(gatesDf):
    gatesDf = gatesDf.dropna(subset=['geometry', 'ref']) 
    newRows = []
    gateNames = []
    offset = 0.0004  
    
    for _, row in gatesDf.iterrows():
        ref = row['ref']
        
        if pd.isna(ref):
            continue  

        if '/' in ref:
            prefix = ''.join(filter(str.isalpha, ref))
            numbers = ref.replace(prefix, '', 1).split('/')
            
            for i, number in enumerate(numbers):
                newRef = f"{prefix}{number.strip()}"  
                newRow = row.copy()
                newRow['ref'] = newRef
                
                if isinstance(newRow.geometry, Point):
                    xOffset = (i + 1) * offset  
                    yOffset = 0  
                    row.geometry = Point(row.geometry.x - 2 * xOffset, row.geometry.y + xOffset / 8)
                    newRow.geometry = Point(newRow.geometry.x + xOffset, newRow.geometry.y + yOffset)
                
                newRows.append(newRow)
                gateNames.append(newRef)  
        else:
            newRows.append(row)
            gateNames.append(ref.strip())
    
    resultDf = pd.DataFrame(newRows)
    resultDf = resultDf[resultDf['ref'].str.lower() != 'nan']
    
    return resultDf, gateNames

# Classify gates into domestic and international categories
def ClassifyGate(ref):
    if pd.notna(ref):
        if ref.startswith('D'):
            return 'Domestic'
        else:
            return 'International'
    return 'Unknown'


##### **Airport Data Filtering:** includes filtering airport features such as aprons and gates.

In [5]:
# Define a bounding box focusing on the terminal areas (International, Central, Domestic)
bbx = (-26.129, -26.137, 28.240, 28.225)  # South, North, East, West

# Download the airport data from OpenStreetMap using the refined bounding box
tag = {'aeroway': True, 'ref': True, 'name': True}  
mdf = ox.features_from_bbox(bbox=bbx, tags=tag)

mdf['ref'] = mdf['ref'].astype(str)
gdf = mdf[~mdf['ref'].str.startswith('B', na=False)]

# Filter for aprons, parking positions, and gates
aprons = gdf[gdf['aeroway'] == 'apron']
parkingPositions = gdf[gdf['aeroway'] == 'parking_position']
gatesP = gdf[gdf['aeroway'] == 'gate']
terminals = gdf[gdf['aeroway'] == 'terminal']
parkingPositions['apronTag'] = parkingPositions['ref'].apply(ClassifyParkingPosition)
parkingPositions = parkingPositions.drop_duplicates(subset='ref', keep='first')

# Define colors for each apron tag
apronColors = {
    'Alpha Apron': 'purple',
    'Charlie Apron': 'cyan',
    'Echo Apron': 'green',
    'Airport Apron': 'magenta'
}

# Initialize dictionary to hold apron positions
apronsPositions = {}
# Group parking positions by apron tag and aggregate unique positions
for apronTag in parkingPositions['apronTag'].unique():
    positions = parkingPositions[parkingPositions['apronTag'] == apronTag]['ref'].tolist()
    apronsPositions[apronTag] = list(set(positions))

# Apply the SplitGates function to the gates DataFrame
gatesP, gateNames = SplitGates(gatesP)

# Add classification to gates DataFrame
gatesP['gateType'] = gatesP['ref'].apply(ClassifyGate)

# Define colors for each gate type
gateColors = {
    'Domestic': 'red',
    'International': 'blue'
}

# Initialize counters and lists
gateCounts = {'Domestic': 0, 'International': 0}
gateLists = {'Domestic': [], 'International': []}

for _, row in gatesP.iterrows():
    gateType = row['gateType']
    ref = row['ref']
    if gateType in gateCounts:
        gateCounts[gateType] += 1
        gateLists[gateType].append(ref)

def ExtractGates(df):
    if 'ref' not in df.columns or 'gateType' not in df.columns:
        raise ValueError("DataFrame must contain 'ref' and 'gateType' columns.")
    
    gateData = {'Domestic': [], 'International': []}
    df = df.dropna(subset=['ref'])
    
    for _, row in df.iterrows():
        gateName = row['ref']
        gateType = row['gateType']
        
        if gateType in gateData:
            gateData[gateType].append(gateName)
        else:
            raise ValueError(f"Invalid gate type: {gateType}. Expected 'Domestic' or 'International'.")

    return gateData

gates = ExtractGates(gatesP)

c:\Users\samra\anaconda3\envs\CV\lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


### **3. Load Flight Data and Map Flight Types**
#### _____________________________________________
#### Load the dataset



In [6]:
# Uncomment and set the file path as needed
filePath = 'test_file_35.csv'
jnbRoutes = pd.read_csv(filePath)

# Define a function to map flight type to the desired output format
def MapFlightType(flightType):
    if 'International' in flightType or 'Arrival' in flightType:
        return 'International'
    elif 'Domestic' in flightType:
        return 'Domestic'
    return 'Unknown'  # Default case if none of the above

# Create the list of flights with the desired format
flights = []
for _, row in jnbRoutes.iterrows():
    flightEntry = {
        'flightId': row['flight_id'],
        'type': MapFlightType(row['type'])
    }
    flights.append(flightEntry)


### **4. Parameters Setup**
#### _____________________________________________
##### In this section, we define several important parameters that will be used throughout the optimization process.



In [7]:
# Set up the number of flights and gates
numFlights = len(flights)
numDomesticGates = len(gates['Domestic'])
numInternationalGates = len(gates['International'])

# Define time slots (assuming hourly slots)
timeSlots = range(24)

# Extract flight IDs and types
flightIds = [f['flightId'] for f in flights]
flightTypes = {f['flightId']: f['type'] for f in flights}

# Extract domestic and international gates
domesticGates = gates['Domestic']
internationalGates = gates['International']

### **4. QUBO Formulation**
#### _____________________________________________
##### **1. Define Binary Variables:** Defines binary variables x, y, and z using PyQubo. These variables represent gate assignments, time slot assignments, and conflict constraints, respectively.


In [8]:
# Define binary variables for gate and time slot assignments
x = {}
for flightId in flightIds:
    for gate in (domesticGates + internationalGates):
        x[flightId, gate] = Binary(f"x_{flightId}_{gate}")

y = {}
for flightId in flightIds:
    for t in range(24):  # Assuming 24 time slots
        y[flightId, t] = Binary(f"y_{flightId}_{t}")

# Auxiliary binary variable for conflicts
z = {}
for gate in (domesticGates + internationalGates):
    for t in range(24):
        z[gate, t] = Binary(f"z_{gate}_{t}")

##### **2. Cost Function:** The cost function is a sum of terms involving these binary variables:

1. ***Gate Assignment:** Ensuring each flight is assigned to exactly one gate and the gate type constraints.*
2. ***Time Slot Assignment:** Ensuring each flight is assigned to exactly one time slot.*
3. ***Conflict Handling:** Handling conflicts by adding penalty terms for the conflict variable.*

##### **3. Quadratic Terms:** (e.g., (gateAssignmentSum - 1) ** 2). 
 

In [9]:
# Define the cost function with penalties
cost = 10
penaltyWeight = 20000  # Adjust this weight as needed

# Ensure each flight is assigned to exactly one gate
for flightId in flightIds:
    flightType = flightTypes[flightId]
    if flightType == 'Domestic':
        gateAssignmentSum = sum(x[flightId, g] for g in domesticGates)
    else:
        gateAssignmentSum = sum(x[flightId, g] for g in internationalGates)
    
    cost += 4 * penaltyWeight * (gateAssignmentSum - 1) ** 2

# Ensure each flight is assigned to at least one gate (domestic + international)
for flightId in flightIds:
    totalGateAssignmentSum = sum(x[flightId, g] for g in (domesticGates + internationalGates))
    cost += 2 * penaltyWeight * (totalGateAssignmentSum - 1) ** 2

# Gate type constraints
for flightId in flightIds:
    flightType = flightTypes[flightId]
    if flightType == 'Domestic':
        for g in internationalGates:
            cost += penaltyWeight * x[flightId, g]
    else:
        for g in domesticGates:
            cost += penaltyWeight * x[flightId, g]

# Time slot assignment
for flightId in flightIds:
    timeSlotAssignmentSum = sum(y[flightId, t] for t in range(24))
    cost += 4 * penaltyWeight * (timeSlotAssignmentSum - 1) ** 2
    
# Constraint to handle conflicts
for gate in (domesticGates + internationalGates):
    for t in range(24):
        cost += penaltyWeight * z[gate, t]

### **5. Formulate and Solve the Model**
#### _____________________________________________
##### This section formulates the optimization model, compiles it into a Binary Quadratic Model (BQM), and uses the Dynex SDK to sample solutions.

In [10]:
model = cost
bqm = model.compile().to_bqm()

In [11]:
# Extract and print interactions
def printBqmInteractions(bqm):
    linear = bqm.linear
    quadratic = bqm.quadratic

    print("Number of Linear interaction: ", len(linear))
    print("Number of Quadratic interaction: ", len(quadratic))

# Print interactions
printBqmInteractions(bqm)

Number of Linear interaction:  2669
Number of Quadratic interaction:  25935


In [12]:
# Initialize Dynex (BQM model and sampler) and  Sample solutions
model = dynex.BQM(bqm)
sampler = dynex.DynexSampler(model,  mainnet=True, description='Dynex SDK Job')
sampleset = sampler.sample(num_reads=10000, annealing_time = 512, debugging=False, is_cluster=True)
bestSolution = sampleset.first.sample
#print('DYNEX RESULT:', bestSolution)

╭────────────┬──────────┬─────────────────┬─────────────┬───────────┬────────────────┬────────────┬─────────┬────────────────╮
│   DYNEXJOB │   QUBITS │   QUANTUM GATES │   BLOCK FEE │   ELAPSED │   WORKERS READ │   CIRCUITS │   STEPS │   GROUND STATE │
├────────────┼──────────┼─────────────────┼─────────────┼───────────┼────────────────┼────────────┼─────────┼────────────────┤
│      51915 │     2677 │           28612 │        0.00 │      3.95 │              1 │      10000 │     512 │   157900000.00 │
╰────────────┴──────────┴─────────────────┴─────────────┴───────────┴────────────────┴────────────┴─────────┴────────────────╯
╭────────────┬─────────────────┬────────────┬───────┬──────────────┬──────────────┬─────────────────────────────┬──────────────┬──────────╮
│     WORKER │         VERSION │   CIRCUITS │   LOC │       ENERGY │      RUNTIME │                 LAST UPDATE │        STEPS │   STATUS │
├────────────┼─────────────────┼────────────┼───────┼──────────────┼──────────────┼──

### **6. Define Extraction Functions**
#### _____________________________________________


In [13]:
# Function to determine gate type
def getGateType(gate):
    if gate in domesticGates:
        return 'Domestic'
    elif gate in internationalGates:
        return 'International'
    return 'Unknown'

# Create a DataFrame to display the results with gate type
def createScheduleDf(gateAssignments, timeSlotAssignments):
    data = []
    for flightId in flightIds:
        gate = gateAssignments.get(flightId, 'None')
        timeSlot = timeSlotAssignments.get(flightId, 'None')
        gateType = getGateType(gate)
        data.append([flightId, gate, timeSlot, gateType])

    df = pd.DataFrame(data, columns=['Flight', 'Gate', 'Time', 'Gate Type'])
    today = datetime.today().strftime('%Y-%m-%d')
    df['Time'] = pd.to_datetime(today + ' ' + df['Time'], format='%Y-%m-%d %H:%M', errors='coerce')
    return df.sort_values(by=['Time', 'Gate Type'], ascending=[True, True])

# Check for gate-time conflicts
def checkGateTimeConflicts(gateAssignments, timeSlotAssignments, conflictAssignments):
    conflicts = {}
    gateTimeAssignments = {}

    for flightId, gate in gateAssignments.items():
        timeSlot = timeSlotAssignments.get(flightId)
        if gate and timeSlot:
            key = (gate, timeSlot)
            gateTimeAssignments.setdefault(key, []).append(flightId)

    formattedConflicts = []
    for (gate, timeSlot), flights in gateTimeAssignments.items():
        if len(flights) > 1 or (gate, timeSlot) in conflictAssignments:
            formattedConflicts.append((gate, timeSlot, flights))

    return formattedConflicts

# Check for mismatches
def checkMismatches(flightIds, gateAssignments, flights):
    mismatches = []
    for flightId in flightIds:
        assignedGate = gateAssignments.get(flightId, None)
        if assignedGate:
            flightType = next((f['type'] for f in flights if f['flightId'] == flightId), 'Unknown')
            if flightType != getGateType(assignedGate):
                mismatches.append((flightId, assignedGate, flightType))
    return mismatches

# Check for unassigned flights
def checkUnassignedFlights(flightIds, gateAssignments, timeSlotAssignments):
    unassignedFlights = []
    for flightId in flightIds:
        gate = gateAssignments.get(flightId, 'None')
        timeSlot = timeSlotAssignments.get(flightId, 'None')
        if gate == 'None' or timeSlot == 'None':
            unassignedFlights.append((flightId, gate, timeSlot))
    return pd.DataFrame(unassignedFlights, columns=['Flight', 'Gate', 'Time'])


### **7. Evaluate FLight Schedule**
#### _____________________________________________


In [14]:
def evaluateSchedule(bestSolution, flightIds, domesticGates, internationalGates, flights):
    allGates = domesticGates + internationalGates
    allTimeSlots = [f"{h:02}:{m:02}" for h in range(24) for m in [0, 30]]  # 24 hours, 2 slots per hour
    # Initialize dictionaries for assignments
    gateAssignments = {}
    timeSlotAssignments = {}
    conflictAssignments = {}

    # Extract gate assignments from bestSolution
    for flightId in flightIds:
        for gate in allGates:
            varName = f"x_{flightId}_{gate}"
            if varName in bestSolution and bestSolution[varName] == 1:
                gateAssignments[flightId] = gate

    # Extract time slot assignments from bestSolution
    for flightId in flightIds:
        for t in range(len(allTimeSlots)):
            varName = f"y_{flightId}_{t}"
            if varName in bestSolution and bestSolution[varName] == 1:
                timeSlotAssignments[flightId] = allTimeSlots[t]

    # Extract conflict indicators from bestSolution
    for gate in allGates:
        for t in range(len(allTimeSlots)):
            varName = f"z_{gate}_{t}"
            if varName in bestSolution and bestSolution[varName] == 1:
                conflictAssignments[(gate, allTimeSlots[t])] = True

    schedule = createScheduleDf(gateAssignments, timeSlotAssignments)
    conflictViolations = checkGateTimeConflicts(gateAssignments, timeSlotAssignments, conflictAssignments)
    mismatches = checkMismatches(flightIds, gateAssignments, flights)
    unassignedFlightsDf = checkUnassignedFlights(flightIds, gateAssignments, timeSlotAssignments)
    unassignedCountBefore = len(unassignedFlightsDf)

    # Reassign flights
    def reassignFlights(conflictViolations, mismatches, unassignedFlightsDf, allGates, allTimeSlots, domesticGates, internationalGates):
        reassignedFlights = []
        availableGatesByTime = {timeSlot: set(allGates) for timeSlot in allTimeSlots}

        # Remove gates already in use at specific times
        for flightId, gate in gateAssignments.items():
            timeSlot = timeSlotAssignments.get(flightId, 'None')
            if timeSlot != 'None' and gate in availableGatesByTime.get(timeSlot, set()):
                availableGatesByTime[timeSlot].remove(gate)

        # Reassign conflicted flights
        for (gate, timeSlot, flightIds) in conflictViolations:
            if not isinstance(flightIds, (list, set)):
                print(f"Expected a list or set for flights, got {type(flightIds)}.")
                continue

            if flightIds:
                firstFlight = flightIds[0]
                if firstFlight in gateAssignments:
                    # Keep the first conflicted flight with its original assignment
                    newGate = gateAssignments[firstFlight]
                    newTime = timeSlotAssignments.get(firstFlight, timeSlot)
                    reassignedFlights.append((firstFlight, newGate, newTime))

                # Reassign the rest of the flights
                for flightId in flightIds[1:]:
                    if flightId in gateAssignments:
                        flightEntry = next((f for f in flights if f['flightId'] == flightId), None)
                        if flightEntry is None:
                            print(f"Flight entry not found for flightId {flightId}.")
                            continue
                        
                        flightType = flightEntry.get('type', 'Unknown')
                        newGate = None
                        if flightType == 'Domestic':
                            newGate = next((g for g in domesticGates if g in availableGatesByTime.get(timeSlot, set())), None)
                        elif flightType == 'International':
                            newGate = next((g for g in internationalGates if g in availableGatesByTime.get(timeSlot, set())), None)
                        
                        if newGate is None:
                            availableSet = availableGatesByTime.get(timeSlot, set())
                            if availableSet:
                                newGate = availableSet.pop()
                            else:
                                print(f"No available gates for time slot {timeSlot}.")
                                continue

                        newTime = timeSlotAssignments.get(flightId, timeSlot)
                        gateAssignments[flightId] = newGate
                        timeSlotAssignments[flightId] = newTime
                        reassignedFlights.append((flightId, newGate, newTime))

                        # Update available gates and times for the specific time slot
                        if newGate and newTime:
                            availableGatesByTime[newTime].discard(newGate)

        # Reassign mismatched flights
        for flightId, gate, flightType in mismatches:
            newGate = None
            timeSlot = timeSlotAssignments.get(flightId, 'None')
            if flightType == 'Domestic':
                newGate = next((g for g in domesticGates if g in availableGatesByTime.get(timeSlot, set())), None)
            elif flightType == 'International':
                newGate = next((g for g in internationalGates if g in availableGatesByTime.get(timeSlot, set())), None)

            if newGate is None:
                availableSet = availableGatesByTime.get(timeSlot, set())
                if availableSet:
                    newGate = availableSet.pop()
                else:
                    print(f"No available gates for time slot {timeSlot}.")
                    continue

            gateAssignments[flightId] = newGate
            timeSlotAssignments[flightId] = timeSlot
            reassignedFlights.append((flightId, newGate, timeSlot))

        # Handle unassigned flights
        for flightId, gate, timeSlot in unassignedFlightsDf.itertuples(index=False):
            if timeSlot == 'None':  # Flight has a gate but no time slot
                # Try to find an available time slot for the assigned gate
                for ts, availableGates in availableGatesByTime.items():
                    if gate in availableGates:
                        newTimeSlot = ts
                        availableGatesByTime[newTimeSlot].remove(gate)
                        timeSlotAssignments[flightId] = newTimeSlot
                        reassignedFlights.append((flightId, gate, newTimeSlot))
                        break
                else:
                    print(f"Failed to assign time slot for flight {flightId} at gate {gate}")
            elif gate == 'None':  # Flight has a time slot but no gate
                if availableGatesByTime.get(timeSlot):
                    newGate = availableGatesByTime[timeSlot].pop()
                    gateAssignments[flightId] = newGate
                    reassignedFlights.append((flightId, newGate, timeSlot))
                else:
                    print(f"Failed to assign gate for flight {flightId} at time {timeSlot}")
            else:
                print(f"Failed to assign gate or time slot for flight {flightId}")

        # Recalculate unassigned flights after reassignment
        print(flightIds)
        unassignedFlightsDf = checkUnassignedFlights(flightIds, gateAssignments, timeSlotAssignments)
        unassignedCountAfter = len(unassignedFlightsDf)
        mismatches = checkMismatches(flightIds, gateAssignments, flights)
        conflictViolations = checkGateTimeConflicts(gateAssignments, timeSlotAssignments, conflictAssignments)
        
        return gateAssignments, timeSlotAssignments, unassignedCountBefore, unassignedCountAfter, mismatches, conflictViolations 

    gateAssignments, timeSlotAssignments, unassignedCountBefore, unassignedCountAfter, mismatches, conflictViolations = reassignFlights(
        conflictViolations, mismatches, unassignedFlightsDf, allGates, allTimeSlots, domesticGates, internationalGates
    )
    # Generate updated schedule DataFrame
    schedule = createScheduleDf(gateAssignments, timeSlotAssignments)
    # Calculate performance
    def calculatePerformance(updatedConflicts, updatedMismatches, unassignedCountAfter):
        numConflicts = len(updatedConflicts)
        numMismatches = len(updatedMismatches)
        totalPossibleConflicts = len(allGates) * len(allTimeSlots)
        performancePercentage = ((totalPossibleConflicts - numConflicts - numMismatches - unassignedCountAfter) / totalPossibleConflicts) * 100 if totalPossibleConflicts > 0 else 0
        performanceMetrics = {
            'Performance Percentage': performancePercentage,
            'Conflicts': numConflicts,
            'Mismatches': numMismatches,
            'Unassigned Flights': unassignedCountAfter
        }
        return performanceMetrics
    
    performanceMetrics = calculatePerformance(conflictViolations, mismatches, unassignedCountAfter)
    return schedule, performanceMetrics


### **8. Display Results**
#### _____________________________________________
1. ***Call the Evaluation Function***
2. ***Display the Flights Schedule:***
3. ***Display Performance Metrics:***



In [15]:
# Call the function to evaluate the schedule
schedule, performanceMetrics = evaluateSchedule(
    bestSolution, flightIds, domesticGates, internationalGates, flights
)

# Display the schedule DataFrame without index
print("\033[1mOR Tambo International Airport Flight Schedule:\033[0m")
# Display DataFrame without index
schedule.style.hide(axis='index')


['SA_4305_JNB_LOS', 'SA_4305_AUH_JNB']
OR Tambo International Airport Flight Schedule:


Flight,Gate,Time,Gate Type
SA_4305_JNB_LHR,A0,2024-09-17 00:00:00,International
SA_4305_JNB_LUN,A7,2024-09-17 00:30:00,International
SA_4305_JNB_MPM,A10,2024-09-17 00:30:00,International
SA_4305_BBK_JNB,A13,2024-09-17 01:00:00,International
SA_4305_MUB_JNB,C10,2024-09-17 01:00:00,International
AI_218_BOM_JNB,A0,2024-09-17 01:30:00,International
SA_4305_ADD_JNB,A10,2024-09-17 01:30:00,International
SA_4305_JNB_LVI,A9,2024-09-17 02:00:00,International
SV_4533_JNB_JED,A2,2024-09-17 02:30:00,International
JE_3393_CPT_JNB,A9,2024-09-17 02:30:00,International


In [16]:

# Display performance metrics
print("\033[1mPerformance Metrics:\033[0m")
print(performanceMetrics)


Performance Metrics:
{'Performance Percentage': 100.0, 'Conflicts': 0, 'Mismatches': 0, 'Unassigned Flights': 0}


# _____________________________________________
# **Ploting**

###  **9: Data Preparation**
#### _____________________________________________


In [17]:
flightsData = schedule
flightsData[['Source', 'Destination']] = flightsData['Flight'].apply(lambda x: pd.Series(ExtractSourceDestination(x)))

#### **Airplane Icon Functions**
#### _____________________________________________


In [18]:
# Load the airplane icon (replace with actual icon loading logic)
def loadAirplaneIcon():
    airplaneIcon = plt.imread('airplane.png')
    return airplaneIcon

def createMaskedIcon(airplaneIcon, color):
    alpha = airplaneIcon[:, :, 3]  
    iconColored = np.zeros_like(airplaneIcon)
    iconColored[:, :, :3] = color  
    iconColored[:, :, 3] = alpha  
    return iconColored

#### **Parking Position Functions**
#### _____________________________________________


In [19]:
def getMidpoints(parkingPositions):
    midpoints = []
    for _, row in parkingPositions.iterrows():
        geometry = row['geometry']
        if isinstance(geometry, LineString):
            midpoint = geometry.interpolate(0.5, normalized=True)
            midpoints.append(midpoint)
    return midpoints

def findClosestParkingPosition(gatePoint, parkingPositions):
    minDistance = float('inf')
    closestPosition = None
    midpoints = getMidpoints(parkingPositions)
    for midpoint in midpoints:
        distanceToGate = gatePoint.distance(midpoint)
        if distanceToGate < minDistance:
            minDistance = distanceToGate
            closestPosition = midpoint
    return closestPosition


#### **Plotting Flights Function**
#### _____________________________________________


In [20]:
def plotFlightsMatplotlib(selectedTime):
    fig, ax = plt.subplots(figsize=(12, 10))  # Increase figure width for legend space
    cmap = plt.get_cmap('viridis')

    # Plot aprons, terminals, parking positions, and gates
    if not aprons.empty:
        for tag, color in apronColors.items():
            apronsTag = aprons[aprons['name'].str.contains(tag, case=False, na=False)]
            if not apronsTag.empty:
                apronsTag.plot(ax=ax, facecolor=color, edgecolor=color, linewidth=1, alpha=0.3, label=f'{tag} (Apron)')

    if not terminals.empty:
        terminals.plot(ax=ax, facecolor="lightgrey", edgecolor="black", linewidth=1, label='Terminals')

    if not parkingPositions.empty:
        for _, row in parkingPositions.iterrows():
            geometry = row.geometry
            ref = row.get('ref', 'N/A')
            apronTag = row['apronTag']

            if isinstance(geometry, LineString):
                midpoint = geometry.interpolate(0.5, normalized=True)
                color = apronColors.get(apronTag, 'grey')
                PlotTriangle(ax, midpoint.x, midpoint.y, color=color)
                if pd.notna(ref):
                    ax.text(midpoint.x, midpoint.y + 0.00005, ref, fontsize=6, ha='center', color='black')

    if not gatesP.empty:
        for _, row in gatesP.iterrows():
            geometry = row['geometry']
            ref = row['ref']
            gateType = row['gateType']
            color = gateColors.get(gateType, 'black')
            if isinstance(geometry, Point):
                ax.plot(geometry.x, geometry.y, color=color, marker='s', markersize=5, label=f'Gates ({gateType})')
                ax.text(geometry.x, geometry.y, ref, fontsize=6, ha='right', color='black')

    # Filter flights based on selected time
    selectedFlights = schedule[schedule['Time'].dt.strftime('%H:%M') == selectedTime]
    if selectedFlights.empty:
        ax.text(0.5, 0.5, 'No Available Flights', fontsize=20, ha='center', va='center', transform=ax.transAxes)
    else:
        flightsGates = selectedFlights.merge(gatesP[['ref', 'geometry']], left_on='Gate', right_on='ref', how='left')
        airplaneIcon = loadAirplaneIcon()
        uniqueFlights = selectedFlights['Flight'].unique()
        colors = cmap(np.linspace(0, 1, len(uniqueFlights)))
        legendHandles = []

        iconWidth, iconHeight = 0.0002, 0.0002

        for i, flight in enumerate(flightsGates.iterrows()):
            _, flightData = flight
            gateGeometry = flightData['geometry']
            if isinstance(gateGeometry, Point):
                gatePoint = gateGeometry
                closestParking = findClosestParkingPosition(gatePoint, parkingPositions)
                if closestParking is not None:
                    midX = (gatePoint.x + closestParking.x) / 2
                    midY = (gatePoint.y + closestParking.y) / 2
                    midpoint = Point(midX, midY)

                    iconBBox = box(midX - iconWidth / 2, midY - iconHeight / 2, midX + iconWidth / 2, midY + iconHeight / 2)

                    insideApron = any(apron['geometry'].contains(iconBBox) for _, apron in aprons.iterrows())
                    insideTerminal = any(terminal['geometry'].contains(iconBBox) for _, terminal in terminals.iterrows())

                    if insideApron and not insideTerminal:
                        flightId = flightData['Flight']
                        gateRef = flightData['Gate']
                        color = colors[i][:3]
                        maskedIcon = createMaskedIcon(airplaneIcon, color)
                        imagebox = OffsetImage(maskedIcon, zoom=0.005)
                        ab = AnnotationBbox(imagebox, (midX, midY+0.0003), frameon=False, pad=0.5, xycoords='data', boxcoords="offset points")
                        ax.add_artist(ab)
                        legendHandles.append(Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=10, label=f'{flightId} (Gate: {gateRef})'))

        ax.set_title(f"Flights at {selectedTime}")
        ax.legend(handles=legendHandles, loc='center left', bbox_to_anchor=(1.05, 0.5))  # Adjust legend position
        ax.set_title("OR Tambo International Airport - Aprons, Parking Positions, Gates, Terminals, and Flights", fontsize=16)

    buffer = io.BytesIO()
    plt.savefig(buffer, format='png', bbox_inches='tight', pad_inches=0.1)  # Ensure tight bounding box to include legend
    plt.close(fig)
    buffer.seek(0)
    imgStr = base64.b64encode(buffer.getvalue()).decode()
    return f'data:image/png;base64,{imgStr}'


### **10. Dash Application**
#### _____________________________________________




In [21]:
app = dash.Dash(__name__)

@app.callback(
    Output('map-global', 'figure'),
    [Input('time-dropdown', 'value')]
)
def updateGlobalMap(selectedTime):
    filteredFlights = flightsData[flightsData['Time'] == selectedTime]

    fig = go.Figure()

    if filteredFlights.empty:
        fig.add_trace(go.Scattergeo(
            lon=[0],
            lat=[0],
            mode='text',
            text='No Available Flights',
            textposition='middle center',
            textfont=dict(size=20, color='black'),
            showlegend=False
        ))
    else:
        for _, flight in filteredFlights.iterrows():
            sourceAirport = flight['Source']
            destinationAirport = flight['Destination']

            sourceData = airportsData[airportsData['airportCode'] == sourceAirport]
            destinationData = airportsData[airportsData['airportCode'] == destinationAirport]

            if not sourceData.empty and not destinationData.empty:
                fig.add_trace(go.Scattergeo(
                    lon=[sourceData['longitude'].values[0]],
                    lat=[sourceData['latitude'].values[0]],
                    mode='markers',
                    marker=dict(size=8, color='blue'),
                    name=f'Source: {sourceAirport}'
                ))

                fig.add_trace(go.Scattergeo(
                    lon=[destinationData['longitude'].values[0]],
                    lat=[destinationData['latitude'].values[0]],
                    mode='markers',
                    marker=dict(size=8, color='red'),
                    name=f'Destination: {destinationAirport}'
                ))

                fig.add_trace(go.Scattergeo(
                    lon=[sourceData['longitude'].values[0], destinationData['longitude'].values[0]],
                    lat=[sourceData['latitude'].values[0], destinationData['latitude'].values[0]],
                    mode='lines',
                    line=dict(width=2, color='green'),
                    name=f'Route: {sourceAirport} to {destinationAirport}'
                ))

    fig.update_layout(
        geo=dict(
            scope='world',
            showland=True,
            landcolor='rgb(212, 212, 212)',
        ),
        title='Flight Paths Between Airports'
    )

    return fig


@app.callback(
    Output('map-airport', 'src'),
    [Input('time-dropdown', 'value')]
)
def updateAirportMap(selectedTime):
    imgStr = plotFlightsMatplotlib(selectedTime)
    return imgStr

### **11. Layout and Execution**
#### _____________________________________________




In [22]:
app.layout = html.Div([
    html.Div([
        dcc.Dropdown(
            id='time-dropdown',
            options=[{'label': f"{i:02d}:00", 'value': f"{i:02d}:00"} for i in range(24)],
            value=flightsData['Time'].dt.strftime('%H:%M').iloc[6]  # Default value
        ),
    ], style={'width': '100%', 'display': 'block', 'margin-bottom': '10px'}),

    html.Div([
        html.Div([
            html.Img(id='map-airport', style={'width': '100%', 'height': '100%', 'object-fit': 'contain'})
        ], style={'flex': '0 0 50%', 'height': '600px', 'margin': '0', 'padding': '0'}),  

        html.Div([
            dcc.Graph(id='map-global', style={'height': '100%'})
        ], style={'flex': '1', 'height': '600px', 'margin': '0', 'padding': '0'})  
    ], style={'display': 'flex', 'height': '600px', 'margin': '0', 'padding': '0'}) 
]) 


In [23]:

if __name__ == '__main__':
    app.run_server(debug=True)


# _____________________________________________